In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import torch

In [3]:
_include_('curriculum_vqa')
_include_('commons')

In [4]:
from cvqa import datasets, model, trainer
from commons import debug

In [8]:
seed = 1

np.random.seed(seed)
torch.manual_seed(seed);

nlvr_root = f'{DEV_HOME}/devdata/nlvr'

In [10]:
train_dataset = datasets.NLVR(nlvr_root, 'dev')
train_dataset

Samples: 989 (N=23)
Vocab Tokens:160

In [7]:
params = {
    'd': 12  # embedding dimension
}

vqa_model = model.build_model(train_dataset.vocab, params)
my_trainer = trainer.Trainer(train_dataset.vocab.pad_index)

In [8]:
vqa_model

TransformerModel(
  (encoder): TransformerEncoder(
    (embed_tokens): Embedding(160, 12, padding_idx=1)
    (embed_positions): SinusoidalPositionalEmbedding()
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (k_proj): Linear(in_features=12, out_features=12, bias=True)
          (v_proj): Linear(in_features=12, out_features=12, bias=True)
          (q_proj): Linear(in_features=12, out_features=12, bias=True)
          (out_proj): Linear(in_features=12, out_features=12, bias=True)
        )
        (self_attn_layer_norm): LayerNorm((12,), eps=1e-05, elementwise_affine=True)
        (fc1): Linear(in_features=12, out_features=2048, bias=True)
        (fc2): Linear(in_features=2048, out_features=12, bias=True)
        (final_layer_norm): LayerNorm((12,), eps=1e-05, elementwise_affine=True)
      )
      (1): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (k_proj): Linear(in_features=12, out_feature

In [8]:
lr = 8e-5
num_epochs=10

optimizer = torch.optim.Adam(vqa_model.parameters(), lr=lr)

my_trainer.train(vqa_model, train_dataset, optimizer)

TypeError: list indices must be integers or slices, not str